# Neural features

In [1]:
import os
import sys 


import pandas as pd
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
import warnings
import torch
from torch.utils.data import random_split

from torch.utils.data import DataLoader
import torch.nn.functional as F 

project_path = os.path.abspath("../code")  # Adatta il percorso a dove si trova il tuo progetto
sys.path.append(project_path)
project_path = os.path.abspath("../networks")  # Adatta il percorso a dove si trova il tuo progetto
sys.path.append(project_path)
from models import *
from vipm_features import *
import vipm_costants as CONST
from vipm_pipeline import *
from dataset import *

def load_csv(csv_path):
    data = pd.read_csv(csv_path, header=None, names=['image_name', 'label'])
    return data['image_name'].tolist(), data['label'].tolist()


In [2]:
csv_path = CONST.SMALL_TRAINING_NAME_PATH
csv_unlabeled = CONST.TRAINING_NAME_PATH
csv_test = CONST.TESTING_NAME_PATH
csv_test_deg = CONST.DEG_TESTING_NAME_PATH
indir_train = CONST.SMALL_TRAINING_PATH  # Modifica in base alla posizione delle immagini
indir_test = CONST.TEST_PATH  # Modifica in base alla posizione delle immagini
indir_deg_test = CONST.DEGRADED_TEST_PATH  # Modifica in base alla posizione delle immagini
outdir = '../dataset/features'  # Modifica in base alla posizione delle feature
# Carica le immagini dal CSV
image_names, labels = load_csv(csv_path)

## KNN

### ResNet50FeatureExtractor

In [5]:

extractor = FeatureExtractor(ResNet50FeatureExtractor())

print(f"\nTesting extractor: {extractor.name}")

try:
    X_train, y_train, _ = extractor.transform(**{"csv":csv_path, "indir":indir_train, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_train.shape[0]} feature di dimensione {X_train.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test, y_test, _ = extractor.transform(**{"csv":csv_test, "indir":indir_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test.shape[0]} feature di dimensione {X_test.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test_deg, y_test_deg, _ = extractor.transform(**{"csv":csv_test_deg, "indir":indir_deg_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test_deg.shape[0]} feature di dimensione {X_test_deg.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

models = [
    KNN(3, standardize=False),
    KNN(11, standardize=False),
    KNN(21, standardize=False),
    KNN(51, standardize=False),
    KNN(3, standardize=True),
    KNN(11, standardize=True),
    KNN(21, standardize=True),
    KNN(51, standardize=True),
]

for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test_deg)
    with warnings.catch_warnings(action="ignore"):
        acc = accuracy_score(y_test_deg, y_pred)
        precision, recall, fscore, _ = precision_recall_fscore_support(y_test_deg, y_pred, average='weighted')
        print(f"Model: KNN - neighbours: {model.n_neighbors} - standardize: {model.standardize}")
        print(f"acc: {acc} - prec: {precision} - rec: {recall} - fscore: {fscore}")



Testing extractor: resnet50
Caricamento delle feature da ../dataset/features/train_small_resnet50_features_normalized.npz
Estratte 5020 feature di dimensione 2048 con resnet50
Caricamento delle feature da ../dataset/features/test_info_resnet50_features_normalized.npz
Estratte 11994 feature di dimensione 2048 con resnet50
Caricamento delle feature da ../dataset/features/test_deg_info_resnet50_features_normalized.npz
Estratte 11994 feature di dimensione 2048 con resnet50
<class 'sklearn.pipeline.Pipeline'>


ValueError: Expected 2D array, got 1D array instead:
array=[247 234 222 ...  72 124 109].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Neural

### ResNet50FeatureExtractor

In [ ]:

extractor = FeatureExtractor(ResNet50FeatureExtractor())

print(f"\nTesting extractor: {extractor.name}")

try:
    X_train, y_train, _ = extractor.transform(**{"csv":csv_path, "indir":indir_train, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_train.shape[0]} feature di dimensione {X_train.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test, y_test, _ = extractor.transform(**{"csv":csv_test, "indir":indir_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test.shape[0]} feature di dimensione {X_test.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")

try:
    X_test_deg, y_test_deg, _ = extractor.transform(**{"csv":csv_test_deg, "indir":indir_deg_test, "outdir":outdir, "normalize":True})
    print(f"Estratte {X_test_deg.shape[0]} feature di dimensione {X_test_deg.shape[1]} con {extractor.name}")
except Exception as e:
    print(f"Errore durante l'elaborazione con {extractor.name}: {e}")


Testing extractor: resnet50
Caricamento delle feature da ../dataset/features/train_small_resnet50_features_normalized.npz
Estratte 5020 feature di dimensione 2048 con resnet50
Caricamento delle feature da ../dataset/features/test_info_resnet50_features_normalized.npz
Estratte 11994 feature di dimensione 2048 con resnet50
Caricamento delle feature da ../dataset/features/test_deg_info_resnet50_features_normalized.npz
Estratte 11994 feature di dimensione 2048 con resnet50


In [11]:
one_layer_model = OneLayerNetwork(3, 251)
one_layer_optimizer = torch.optim.Adam(one_layer_model.parameters(), lr=0.01)
one_layer_scheduler = torch.optim.lr_scheduler.StepLR(one_layer_optimizer, step_size=5, gamma=0.1)
one_layer_model_option = ModelOptions(torch.nn.CrossEntropyLoss(), one_layer_optimizer, one_layer_scheduler, input_dim = 3)
nn = NeuralNetwork(one_layer_model, one_layer_model_option)

training_set = FeatureDataset(f"{outdir}/train_small_rgb_mean_features_normalized.npz",
                              type='test',
                              target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

test_set = FeatureDataset(f"{outdir}/test_info_rgb_mean_features_normalized.npz",
                          type='test',
                          target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

test_set_degraded = FeatureDataset(f'{outdir}/test_deg_info_rgb_mean_features_normalized.npz',
                                type='test',
                                target_transform=lambda y: F.one_hot(y, num_classes=one_layer_model_option.num_classes))

train_size = int(0.8 * len(training_set))  # 80% for training
val_size = len(training_set) - train_size  # Remaining 20% for validation
training_set, val_dataset = random_split(training_set, [train_size, val_size], torch.Generator().manual_seed(42))

train_loader = DataLoader(training_set, batch_size=one_layer_model_option.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=one_layer_model_option.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=one_layer_model_option.batch_size, shuffle=False)
test_degraded_loader = DataLoader(test_set_degraded, batch_size=one_layer_model_option.batch_size, shuffle=False)

nn.fit(train_loader, val_loader)
mean_loss, accuracy, y_pred, y_test = nn.predict(test_loader)

with warnings.catch_warnings(action="ignore"):
    acc = accuracy_score(y_test, y_pred)
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print(f"acc: {acc} - prec: {precision} - rec: {recall} - fscore: {fscore}")


Epoch 1/100:
  Train Loss: 5.6562, Train Accuracy: 0.35%
  Val Loss: 5.5278, Val Accuracy: 0.30%
Epoch 2/100:
  Train Loss: 5.5823, Train Accuracy: 0.50%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 3/100:
  Train Loss: 5.5325, Train Accuracy: 0.75%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 4/100:
  Train Loss: 5.4835, Train Accuracy: 0.85%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 5/100:
  Train Loss: 5.4339, Train Accuracy: 0.97%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 6/100:
  Train Loss: 5.3906, Train Accuracy: 1.12%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 7/100:
  Train Loss: 5.3982, Train Accuracy: 1.44%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 8/100:
  Train Loss: 5.3770, Train Accuracy: 1.47%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 9/100:
  Train Loss: 5.3794, Train Accuracy: 1.32%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 10/100:
  Train Loss: 5.3655, Train Accuracy: 1.12%
  Val Loss: 5.5344, Val Accuracy: 0.20%
Epoch 11/100:
  Train Loss: 5